# Webscraping demo
## Tripadvisor hotel reviews
### This notebook uses the Excel file *HotelsToScrap.xlsx* as the list of hotels to scrap
#### Changes may be required due to Tripadvisor's continous updates


### Load packages and do the initializations

In [1]:
# Load libraries
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time

In [2]:
# Allow not verified SSL (Secure Socket Layer) certificates to be opened
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# Get Firefox options (configurations)
options = Options()

# Add this argument to Options to avoid being detected as a robot
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")


# Add this argument to Options to hide Firefox (make it not visible)
options.add_argument('--headless') 

In [4]:
# Load the list of the hotels to read the content
hotelsToScrap = pd.read_excel("HotelsToScrap.xlsx", sheet_name="Sheet1", index_col="ID", engine='openpyxl')
hotelsToScrap.index = ["Aloft_Miami", "Hampton Inn", "Myriad", "AvenidaPalace", "Envue"]
hotelsToScrap.URL = ['https://www.tripadvisor.co.uk/Hotel_Review-g34438-d10822168-Reviews-Aloft_Miami_Airport-Miami_Florida.html',
       'https://www.tripadvisor.co.uk/Hotel_Review-g34792-d10746316-Reviews-Hampton_Inn_Suites_Braselton-Braselton_Georgia.html',
       'https://www.tripadvisor.co.uk/Hotel_Review-g189158-d3323582-Reviews-Myriad_by_SANA_Hotels-Lisbon_Lisbon_District_Central_Portugal.html',
       'https://www.tripadvisor.co.uk/Hotel_Review-g189158-d195643-Reviews-Hotel_Avenida_Palace-Lisbon_Lisbon_District_Central_Portugal.html',
       'https://www.tripadvisor.co.uk/Hotel_Review-g46907-d17721483-Reviews-Envue_Autograph_Collection-Weehawken_New_Jersey.html']


In [7]:
# Create an empty dataframe for the resuls
#review date ver qual e a specific variable
hotelReviews = pd.DataFrame({'hotelID': pd.Series([], dtype='string'),
                             'user': pd.Series([], dtype='string'),
                             'rating': pd.Series([], dtype='float'),
                             'text': pd.Series([], dtype='string'),
                             'date': pd.Series([], dtype='string'), 
                             'location': pd.Series([], dtype='string')
                             })

### Functions to use in the Main Loop

In [9]:
# Open page and read HTML
def openPageReadHTML(url):

    # Open Firefox with Selenium
    #binary = FirefoxBinary('/Applications/Firefox.app/Contents/MacOS/firefox-bin')  # in case of error, replace the Firexfox path with the one on your computer
    #browser = webdriver.Firefox(firefox_binary=binary, options=options)
    browser = webdriver.Safari(options=options)
    browser.get(url)
    time.sleep(1) # Wait one second

    # If there is a privacy pop-up, click the OK button
    privacy_button = browser.find_elements(By.ID,"onetrust-accept-btn-handler")
    if len(privacy_button)>0:
        browser.execute_script("arguments[0].click()", privacy_button[0])
        time.sleep(0.5) # Wait half a second

    # Try to move into first review and click the button 
    # Some times it takes some time to load the page
    clicked_button=False
    while not clicked_button:
        read_more_buttons = browser.find_elements(By.CLASS_NAME,"eljVo")
        if len(read_more_buttons) > 0:
            try: 
                browser.execute_script("arguments[0].scrollIntoView(true);", read_more_buttons[0])
                browser.execute_script("arguments[0].click()", read_more_buttons[0])
                time.sleep(0.5) # Wait half a second
                clicked_button=True
            except:
                # Wait for one second to retry
                time.sleep(1)
        else:
            # Wait for one second to retry
            time.sleep(1)
    
    # Read the content close de browser
    html_source = browser.page_source  
    browser.quit()

    # Transform the html into a BeautifulSoup object
    soupObj = BeautifulSoup(html_source) 

    return soupObj

In [10]:
import pandas as pd
import datetime as dt
import re

In [13]:
# Process each page
def processPage(soupObj, hotelID, extractedDF):

    # Read reviews
    reviews = soupObj.find_all("div", {"data-test-target": "reviews-tab"})
    

    # Get the list of reviews
    reviewsList = reviews[0].select("div[class*=cWwQK]")
    # Loop thru each review
    for i in range(0,len(reviewsList)):
        # Get Rating
        r1 = reviewsList[i].select("span[class*=ui_bubble_rating]")
        r2 = r1[0]["class"][1]
        reviewRating = int(''.join(filter(str.isdigit, r2)))/10

        # Get User
        user = reviewsList[i].select("a[class*=ui_header_link]")[0].string

        # Get review text
        t = reviewsList[i].select("q[class*=XllAv]")[0]
        reviewText = t.get_text()
        
        
        # Get review date 
        reviewDate = reviewsList[i].select("div[class*=bcaHz]")[0].get_text()
        reviewDate = reviewDate.split()
        reviewDate = reviewDate[-2] + " " + reviewDate[-1]
        print(reviewDate)
        
        if reviewDate =='review Today':reviewDate = dt.datetime.today()
       
        elif re.search(r'[S]\d{3}',reviewDate) == 'Sept':
            reviewDate2 = reviewDate.replace('Sept', 'Sep')
        
        elif reviewDate =='review Yesterday':
            reviewDate = dt.datetime.today()+dt.timedelta(days=-1)

        elif re.search(r'[12]\d{3}',reviewDate) is None:
            reviewDate2 = reviewDate + ' ' + str(dt.datetime.today().year)
            reviewDate = dt.datetime.strptime(reviewDate2, "%d %b %Y")

        else:
            reviewDate2 = '1 '+ reviewDate
            reviewDate = dt.datetime.strptime(reviewDate2, "%d %b %Y")
        
        
        # Print the date as a string
        reviewDatestr = reviewDate.strftime("%d/%m/%Y")

        
        
        # Get location  
        location = reviewsList[i].select("span[class*=fSiLz]")
        if len(location)>0:
            location = location[0].get_text()
        else:
            location ='N/A'                               
                            
              
        # Update extracted reviews dataframe
        extractedDF = extractedDF.append({'hotelID': hotelID,
                             'user': user,
                             'rating': reviewRating,
                             'text': reviewText,
                             'date': reviewDate,
                             'location': location
                             }, ignore_index=True)

    # Return the resulting dataframe
    return extractedDF

### Main loop

In [14]:
proxy = "124.240.187.80:82"

webdriver.DesiredCapabilities.SAFARI['proxy'] = {
   "httpProxy":proxy,
   "ftpProxy":proxy,
   "sslProxy":proxy,
   "noProxy":None,
   "proxyType":"MANUAL",
   "class":"org.openqa.selenium.Proxy",
   "autodetect":False
}

In [18]:
# Loop for all hotels
for index, row in hotelsToScrap.iterrows():
    if index == "Aloft_Miami":
        reviewsToGet = 200
    if index == "Myriad":
        continue
    if index == "Hampton Inn":
        continue
    if index == "AvenidaPalace":
        continue
    if index == "Envue":
        continue
    if index == "Corinthia":
        continue
    # Present feedback on which hotel is being processed
    print("Processing hotel", index)

    # Reset counter per hotel
    reviewsExtracted = 0    

    # Loop until it extracts the pre-defined number of reviews
    while reviewsExtracted<reviewsToGet:

        # Define URL to use based on the number of reviews extracted so far
        urlToUse = row['URL']
        if reviewsExtracted>0:
            repText = "-Reviews-or"+str(reviewsExtracted)+"-"
            urlToUse = urlToUse.replace("-Reviews-",repText)

        # Open and read the web page content
        soup = openPageReadHTML(urlToUse)

        # Process web page
        hotelReviews = processPage(soup, index, hotelReviews)

        # Update counter
        reviewsExtracted = reviewsExtracted + 5

        # Present feedback on the number of extracted reviews
        print("Extracted ",reviewsExtracted,"/",reviewsToGet)
     

Processing hotel Aloft_Miami
May 2022
May 2022
May 2022
May 2022
May 2022
Mar 2022
Mar 2022
Mar 2022
Feb 2022
Feb 2022
Extracted  5 / 200
Mar 2022
Mar 2022
Mar 2022
Feb 2022
Feb 2022
Dec 2021
Nov 2021
Oct 2021
Oct 2021
Oct 2021
Extracted  10 / 200
Dec 2021
Nov 2021
Oct 2021
Oct 2021
Oct 2021
Sep 2021
Sep 2021
Aug 2021
Aug 2021
Aug 2021
Extracted  15 / 200
Sep 2021
Sep 2021
Aug 2021
Aug 2021
Aug 2021
Aug 2021
Aug 2021
Jul 2021
Jul 2021
Jul 2021
Extracted  20 / 200
Aug 2021
Aug 2021
Jul 2021
Jul 2021
Jul 2021
Jul 2021
May 2021
Mar 2021
Feb 2021
Jan 2021
Extracted  25 / 200
Jul 2021
May 2021
Mar 2021
Feb 2021
Jan 2021
Jan 2021
Dec 2020
Aug 2020
Aug 2020
Mar 2020
Extracted  30 / 200
Jan 2021
Dec 2020
Aug 2020
Aug 2020
Mar 2020
Mar 2020
Mar 2020
Feb 2020
Feb 2020
Feb 2020
Extracted  35 / 200
Mar 2020
Mar 2020
Feb 2020
Feb 2020
Feb 2020
Feb 2020
Jan 2020
Jan 2020
Jan 2020
Dec 2019
Extracted  40 / 200
Feb 2020
Jan 2020
Jan 2020
Jan 2020
Dec 2019
Dec 2019
Dec 2019
Dec 2019
Dec 2019
Dec 2019
Ex

In [20]:
hotelReviews.to_csv("Aloft_Miami.csv", index = False)

In [25]:
hotelReviews.to_csv("HamptonInn.csv", index = False)

In [16]:
hotelReviews.to_csv("envue_full.csv", index = False)

In [87]:
# Using lambda function in order to convert the date in the extracted file.
# Save the extracted reviews data frame to an Excel file
hotelReviews['date'] = hotelReviews['date'].apply(lambda x: pd.to_datetime(x))
hotelReviews['date'] = hotelReviews['date'].dt.date
hotelReviews
hotelReviews.to_excel("ExtractedReviewsTESTE2.xlsx")

In [20]:
sheraton_reviews.to_csv("sheraton.csv", index = False)

In [88]:
hotelReviews.groupby("hotelID")["date"].min()

hotelID
AvenidaPalace    2020-03-01
Corinthia        2021-06-01
Mar Mante        2021-06-01
Myriad           2019-09-01
Name: date, dtype: object

In [90]:
myriad_reviews = hotelReviews.loc[hotelReviews.hotelID == "AvenidaPalace "]

In [92]:
myriad_reviews.to_csv("myriad_reviews.csv", index= False)

In [95]:
avenida_palace_reviews = hotelReviews.loc[hotelReviews.hotelID == "AvenidaPalace"]

In [96]:
avenida_palace_reviews.to_csv("avenida_palace_reviews.csv", index = False)